In [ ]:
from dotenv import load_dotenv
import os
from IPython.display import display, Markdown, Latex
from langchain_openai import ChatOpenAI
import warnings
from src.youtube_sommelier.tools.youtube import SumarizeVideo, SearchTranscript
from youtube_sommelier.crew import YoutubeSommelierCrew

load_dotenv()

warnings.filterwarnings("ignore")

model_name = os.getenv("OPENAI_MODEL_NAME", "gpt-4o-mini")
llm = ChatOpenAI(model_name=model_name, temperature=0)

In [ ]:
#input_text  = "https://www.youtube.com/watch?v=VsZXAEFKXC4"
input_text = "https://www.youtube.com/watch?v=GIXmIAbrHOc"

inputs = {
    "topic": input_text
}

In [ ]:
result = YoutubeSommelierCrew().crew().kickoff(inputs=inputs)

In [ ]:
Markdown(result.raw)

In [ ]:
test_tool = SearchTranscript().run(
    url_video=input_text, 
    human_input="Léo compartilha histórias de censura em várias cidades do Brasil."
)

Markdown(test_tool)

In [ ]:
test_tool = SearchTranscript().run(
    url_video=input_text, 
    human_input="Léo Lins anuncia o lançamento de seu livro"
)

Markdown(test_tool)